In [5]:
import re

file = open('input1.txt', 'r')
regex = '(1|2|3|4|5|6|7|8|9|0|one|two|three|four|five|six|seven|eight|nine)'
translation = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

sum_digits = 0
sum_digits_and_text = 0

for line in file.readlines():
    matches = []
    for i in range(len(line)):
        match = re.search(regex, line[i:])
        if match is not None:
            matches.append(match.group(0))

    first = -1
    first_digit = -1

    last = -1
    last_digit = -1

    for match in matches:
        digit = 0
        try:
            digit = int(match)
            if first_digit == -1:
                first_digit = digit
            last_digit = digit
        except ValueError:
            digit = translation[match]

        if first == -1:
            first = digit
        last = digit

    if first == -1 or last == -1:
        print(matches)
        print(first)
        print(last)
        
    sum_digits += 10*first_digit + last_digit
    sum_digits_and_text += 10*first + last

print(f'The sum of the first and last digit on each line is {sum_digits}')
print(f'The sum of the first and last digit on each line (including text) is {sum_digits_and_text}')
        

The sum of the first and last digit on each line is 53386
The sum of the first and last digit on each line (including text) is 53312


In [6]:
import re
file = open('input2.txt')

max_red = 12
max_green = 13
max_blue = 14

impossible = []
possible = []
powers = []

for line in file.readlines():
    game_match = re.match('Game ([0-9]+): (.+)$', line)
    id = int(game_match.group(1))
    draws = game_match.group(2).split(';')

    min_red = 0
    min_green = 0
    min_blue = 0
    
    for draw in draws:
        red_match = re.search('([0-9]+) red', draw)
        green_match = re.search('([0-9]+) green', draw)
        blue_match = re.search('([0-9]+) blue', draw)

        red = 0 if red_match is None else int(red_match.group(1))
        green = 0 if green_match is None else int(green_match.group(1))
        blue = 0 if blue_match is None else int(blue_match.group(1))

        min_red = red if red > min_red else min_red
        min_green = green if green > min_green else min_green
        min_blue = blue if blue > min_blue else min_blue

        if id not in impossible and (red > max_red or green > max_green or blue > max_blue):
            impossible.append(id)

    if id not in impossible:
        possible.append(id)

    powers.append(min_red*min_green*min_blue)

print(f'The sum of the IDs of the games that were possible is {sum(possible)}')
print(f'The sum of the powers of the minimum sets of cubes is {sum(powers)}')
        

The sum of the IDs of the games that were possible is 2406
The sum of the powers of the minimum sets of cubes is 78375


In [7]:
import re
import numpy

file = open('input3.txt')
schematic = []
part_numbers = []

gears = {}

for line in file.readlines():
    schematic.append(line.strip())

running_number = ''
adjacent_gears = set()
adjacent = False

for i in range(len(schematic)):
    for j in range(len(schematic[i])):
        c = schematic[i][j]
        if c.isdigit():
            running_number += c

            # Check nearby cells for symbols
            for k in range(-1, 2):
                for l in range(-1, 2):
                    try:
                        other_c = schematic[i+k][j+l]
                        if re.search('[^\.0-9]', other_c) is not None:
                            adjacent = True
                        if other_c == '*':
                            adjacent_gears.add((i+k, j+l))
                    except IndexError:
                        pass

        else:
            if adjacent:
                part_numbers.append(int(running_number))
                for gear in adjacent_gears:
                    if gear not in gears:
                        gears[gear] = []
                    gears[gear].append(int(running_number))

            running_number = ''
            adjacent_gears = set()
            adjacent = False

print(f'The sum of all the part numbers is {sum(part_numbers)}')
print(f'The sum of all gear ratios is {sum(numpy.prod(gears[gear]) for gear in gears if len(gears[gear]) == 2)}')

The sum of all the part numbers is 539637
The sum of all gear ratios is 82818007


In [8]:
import re

file = open('input4.txt', 'r')
cards = []
for line in file.readlines():
    match = re.search('Card +([0-9]+): +(.*)', line)
    winning_numbers, card_numbers = match.group(2).split('|')

    winning_numbers = winning_numbers.split(' ')
    winning_numbers = [int(number) for number in winning_numbers if number.isdigit()]

    card_numbers = card_numbers.split(' ')
    card_numbers = [int(number) for number in card_numbers if number.isdigit()]

    cards.append({
        'winning_numbers': winning_numbers,
        'card_numbers': card_numbers
    })

points = 0
for card in cards:
    count = 0
    for number in card['card_numbers']:
        if number in card['winning_numbers']:
            count += 1
    if count > 0:
        points += 2**(count-1)

    card['count'] = count

print(f'The sum of scratch card points is {points}')

number_of_cards = [1 for card in cards]
for i in range(len(cards)):
    count = cards[i]['count']
    for j in range(count):
        number_of_cards[i+j+1] += number_of_cards[i]

print(f'The total number of scratch cards at the end is {sum(number_of_cards)}')

The sum of scratch card points is 19135
The total number of scratch cards at the end is 5704953


In [4]:
import re

file = open('input5.txt', 'r')
contents = file.read()

map_names = [
    'seed-to-soil',
    'soil-to-fertilizer',
    'fertilizer-to-water',
    'water-to-light',
    'light-to-temperature',
    'temperature-to-humidity',
    'humidity-to-location'
]

def create_map(input):
    map = []
    
    lines = input.split('\n')
    for line in lines:
        try:
            target, source, range = line.split(' ')
            map.append((int(source), int(target), int(range)))
        except ValueError:
            pass
    return map

def apply_map(number, map):
    for mapping in map:
        source, target, range = mapping
        if 0 <= number - source < range:
            return target + (number - source)
    return number

seeds = [int(x.strip()) for x in re.search('seeds: (.+)\n', contents).group(1).split(' ')]
maps = {map_name: create_map(re.search(f'{map_name} map:\n((.+\n)+)\n*', contents).group(1)) for map_name in map_names}

locations = []
for seed in seeds:
    number = seed
    for map_name in map_names:
        number = apply_map(number, maps[map_name])
    locations.append(number)
print(f'The closest location for the given seeds is {min(locations)}')

# Brute force
def apply_map_reverse(number, map):
    for mapping in map:
        source, target, range = mapping
        if 0 <= number - target < range:
            return source + (number - target)
    return number

def check_if_in_seeds(number, seeds):
    for i in range(0, len(seeds), 2):
        start = seeds[i]
        length = seeds[i+1]

        if 0 <= number - start <= length:
            return True

    return False

for i in range(min(locations)):
    number = i
    for map_name in map_names[::-1]:
        number = apply_map_reverse(number, maps[map_name])

    if i % 10000 == 0:
        print(f'\rSearched through {i}', end='')

    if check_if_in_seeds(number, seeds):
        print()
        print(f'Lowest location is {i}, corresponding to seed {number}')
        break

The closest location for the given seeds is 662197086
Searched through 52510000
Lowest location is 52510809, corresponding to seed 3107439671


In [41]:
import numpy
import math

file = open('input6.txt', 'r')
times_string, distances_string = file.readlines()
times = [int(x) for x in times_string.split() if x.isdigit()]
distances = [int(x) for x in distances_string.split() if x.isdigit()]

wins = []
for i in range(len(times)):
    current_wins = 0
    time = times[i]
    record = distances[i]

    for charge in range(time):
        if time*charge - charge**2 > record:
            current_wins += 1
    wins.append(current_wins)

print(f'The product of the numbers of ways to win the short races is {numpy.prod(wins)}')

# One long race
time = int(''.join(x for x in times_string if x.isdigit()))
record = int(''.join(x for x in distances_string if x.isdigit()))

best_race = time / 2
match_record_low = (time - math.sqrt(time**2 - 4 * record)) / 2
match_record_high = (time + math.sqrt(time**2 - 4 * record)) / 2
first_better = math.floor(match_record + 1)
last_better = math.ceil(match_record_high - 1)

print(f'The number of ways to win the really long race is {last_better - first_better + 1}')

The product of the numbers of ways to win the short races is 1624896
The number of ways to win the really long race is 32583852
